In [ ]:
!unzip "/content/drive/MyDrive/Colab Notebooks/curso word2vec/cbow_s300.zip"
!unzip "/content/drive/MyDrive/Colab Notebooks/curso word2vec/skip_s300.zip"

Archive:  /content/drive/MyDrive/Colab Notebooks/curso word2vec/cbow_s300.zip
  inflating: cbow_s300.txt           
Archive:  /content/drive/MyDrive/Colab Notebooks/curso word2vec/skip_s300.zip
  inflating: skip_s300.txt           


## Libs Usadas

In [ ]:
import nltk
import string
import numpy as np
import pandas as pd
from gensim.models import KeyedVectors
from sklearn.dummy import DummyClassifier
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer

nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

## Leitura dos dados

In [ ]:
artigo_treino = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/curso word2vec/treino.csv')
artigo_teste = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/curso word2vec/teste.csv')

In [ ]:
display(artigo_treino.head())
display(artigo_treino.shape)
display(artigo_teste.head())
display(artigo_teste.shape)

,title,text,date,category,subcategory,link
0,"Após polêmica, Marine Le Pen diz que abomina n...",A candidata da direita nacionalista à Presidên...,2017-04-28,mundo,NaN,http://www1.folha.uol.com.br/mundo/2017/04/187...
1,"Macron e Le Pen vão ao 2º turno na França, em ...",O centrista independente Emmanuel Macron e a d...,2017-04-23,mundo,NaN,http://www1.folha.uol.com.br/mundo/2017/04/187...
2,"Apesar de larga vitória nas legislativas, Macr...",As eleições legislativas deste domingo (19) na...,2017-06-19,mundo,NaN,http://www1.folha.uol.com.br/mundo/2017/06/189...
3,"Governo antecipa balanço, e Alckmin anuncia qu...",O número de ocorrências de homicídios dolosos ...,2015-07-24,cotidiano,NaN,http://www1.folha.uol.com.br/cotidiano/2015/07...
4,"Após queda em maio, a atividade econômica sobe...","A economia cresceu 0,25% no segundo trimestre,...",2017-08-17,mercado,NaN,http://www1.folha.uol.com.br/mercado/2017/08/1...


(90000, 6)

,title,text,date,category,subcategory,link
0,Grandes irmãos,"RIO DE JANEIRO - O Brasil, cada vez menos famí...",2017-03-06,colunas,ruycastro,http://www1.folha.uol.com.br/colunas/ruycastro...
1,Haddad congela orçamento e suspende emendas de...,"O prefeito de São Paulo, Fernando Haddad (PT),...",2016-08-10,colunas,monicabergamo,http://www1.folha.uol.com.br/colunas/monicaber...
2,Proposta de reforma da Fifa tem a divulgação d...,"A Fifa divulgou, nesta quinta (10), um relatór...",2015-10-09,esporte,NaN,http://www1.folha.uol.com.br/esporte/2015/09/1...
3,"Mercado incipiente, internet das coisas conect...","Bueiros, coleiras, aparelhos hospitalares, ele...",2016-11-09,mercado,NaN,http://www1.folha.uol.com.br/mercado/2016/09/1...
4,"Mortes: Psicanalista, estudou o autismo em cri...",Toda vez que o grupo de amigos de Silvana Rabe...,2017-02-07,cotidiano,NaN,http://www1.folha.uol.com.br/cotidiano/2017/07...


(20513, 6)

## Estudo de como funciona o Word2Vec

In [ ]:
modelo = KeyedVectors.load_word2vec_format('/content/cbow_s300.txt')

In [ ]:
modelo.most_similar('carro')

[('veículo', 0.8137416839599609),
 ('caminhão', 0.7600934505462646),
 ('jipe', 0.680985689163208),
 ('camião', 0.6723827719688416),
 ('furgão', 0.6717828512191772),
 ('veã\xadculo', 0.6359531879425049),
 ('caminhã£o', 0.620665967464447),
 ('avião', 0.610375165939331),
 ('táxi', 0.60887610912323),
 ('automóvel', 0.6054151058197021)]

In [ ]:
modelo.most_similar(positive=["brasil", "argentina", "uruguai"])

[('chile', 0.7778865098953247),
 ('peru', 0.7384953498840332),
 ('paraguai', 0.7282066345214844),
 ('equador', 0.6968042850494385),
 ('bolívia', 0.6627143621444702),
 ('méxico', 0.658368706703186),
 ('haiti', 0.6435114741325378),
 ('panamá', 0.6385565400123596),
 ('venezuela', 0.6315447688102722),
 ('colômbia', 0.6180727481842041)]

In [ ]:
'''
Possível maneira de descobrir o plural de palavras.

nuvens -> nuvem
estrelas -> estrelas

Como descobrir o plural da palavra "estrelas" com base nas outras informações?

nuvens + estrela - nuvem = estrelas

As operações vetoriais deveriam resultar assim.
'''

modelo.most_similar(positive=["nuvens", "estrela"], negative=["nuvem"])

[('estrelas', 0.5497429966926575),
 ('plêiades', 0.379197895526886),
 ('colinas', 0.3746805191040039),
 ('trovoadas', 0.373703271150589),
 ('sombras', 0.3734194040298462),
 ('pombas', 0.3726757764816284),
 ('corredoras', 0.3640727698802948),
 ('cigarras', 0.36065393686294556),
 ('galáxias', 0.35754913091659546),
 ('luas', 0.3575345277786255)]

In [ ]:
'''
Nem tudo funciona perfeitamente.

Nesse caso abaixo podemos ver o claro vies humano.
Onde é associado a figura da mulher a enfermeira em vez de médica.
'''

modelo.most_similar(positive=["médico", "mulher"], negative=["homem"])

[('enfermeira', 0.6180862188339233),
 ('psicóloga', 0.47447532415390015),
 ('dama-de-companhia', 0.4738234877586365),
 ('esposa', 0.46559134125709534),
 ('parteira', 0.46362876892089844),
 ('mãe', 0.4581768214702606),
 ('governanta', 0.45722925662994385),
 ('madrasta', 0.4569782614707947),
 ('menina', 0.44435906410217285),
 ('filha', 0.44341108202934265)]

In [ ]:
'''
Para um caso em que não temos tanto um vies humano, a relação funciona
corretamente, pois nesse caso é mais comum associar a mulher o cargo
de professora.

Assim, é muito importante fazer um estudo dos dados que foram analisados.
'''

modelo.most_similar(positive=["professor", "mulher"], negative=["homem"])

[('professora', 0.6192208528518677),
 ('aluna', 0.5449554324150085),
 ('esposa', 0.4978231191635132),
 ('ex-aluna', 0.4884248375892639),
 ('namorada', 0.4737858772277832),
 ('enfermeira', 0.4728144109249115),
 ('filha', 0.467373788356781),
 ('irmã', 0.45845913887023926),
 ('ex-namorada', 0.45824766159057617),
 ('ex-professora', 0.4510470926761627)]

## Tokenização
#### Vetorização de textos

In [ ]:
artigo_treino.title.loc[8]

'Premiada em Sundance, Crystal Moselle retrata sexismo no mundo do skate'

In [ ]:
'''
Função que tokeniza um texto separando pela pontuação.

Recebe uma frase como entrada.
'''

def tokenizador(texto):
  lista_alfanumerica = list()

  texto_min = texto.lower()
  
  for token_valido in nltk.word_tokenize(texto_min):
    if token_valido in string.punctuation: continue
    lista_alfanumerica.append(token_valido)

  return lista_alfanumerica

In [ ]:
'''
Função que cria e ajusta a frase de entrada para um vetor.

Recebe uma frase que já foi tokenizada como entrada.
'''

def combinacao_vetores_soma(palavras_numeros):
  vetor_resultante = np.zeros(300)

  for pn in palavras_numeros:
    try:
      vetor_resultante += modelo.get_vector(pn)
    except KeyError:
      if pn.isnumeric():
        pn = '0'*len(pn)
        vetor_resultante += modelo.get_vector(pn)
      else:
        vetor_resultante += modelo.get_vector('unknown')

  return vetor_resultante

In [ ]:
'''
Ainda existem erros nesse algoritmo.

Como o fato de palavras escritas erradas gerarem erro no Word2Vec.

Também temos o problema dos números, onde é preciso transformar
todos para zero.

Isso foi descrito no artigo científico do projeto do Word2Vec que
estamos usando.

As correções foram feitas na função acima com o try : except.
'''

palavras_numeros = tokenizador("texto alura")
vetor_texto = combinacao_vetores_soma(palavras_numeros)
print(len(vetor_texto))
print(vetor_texto)

300
[ 0.125897    0.194438   -0.142889    0.34635099  0.035276   -0.017386
  0.030712    0.166557    0.045682   -0.062929    0.06109     0.028584
 -0.138883   -0.060791    0.02032     0.00423    -0.279804    0.02043
  0.096948    0.121195   -0.19849801 -0.114145   -0.11284101  0.002815
  0.23597    -0.090686   -0.088413    0.078109   -0.013849    0.102817
  0.148391   -0.285559   -0.011405   -0.066368    0.100877    0.116555
  0.11000501  0.30112999  0.084156    0.33384     0.015387   -0.066228
  0.22636401  0.014496   -0.015116    0.101645    0.154648    0.10974901
  0.042056    0.027007    0.171579    0.062411   -0.25566     0.026175
 -0.147822   -0.106932    0.053303    0.31571199 -0.005591   -0.314451
 -0.25245201 -0.06189    -0.131413   -0.195418    0.12041599 -0.110805
  0.147487   -0.06038    -0.15548899 -0.072362    0.061702   -0.27237701
  0.036561    0.059108   -0.038358    0.154295   -0.130129   -0.220903
  0.113842   -0.032527    0.03557    -0.143014    0.252334    0.098097

In [ ]:
'''
Função para transformar o conjunto de treino e teste em uma
matriz de vetores.

Recebe o conjunto de textos do DataFrame como entrada.
'''

def matriz_vetores(textos):
  x = len(textos)
  y = 300
  matriz = np.zeros((x, y))

  for i in range(x):
    palavras_numeros = tokenizador(textos.iloc[i])
    matriz[i] = combinacao_vetores_soma(palavras_numeros)
  
  return matriz

In [ ]:
matriz_vetores_treino = matriz_vetores(artigo_treino.title)
matriz_vetores_teste = matriz_vetores(artigo_teste.title)

# Treinamento e Comparações

## Treinamento com CBOW

In [ ]:
print(matriz_vetores_treino.shape)
print(matriz_vetores_teste.shape)

(90000, 300)
(20513, 300)


### Regressão Logística

In [ ]:
lr = LogisticRegression(max_iter=200)
lr.fit(matriz_vetores_treino, artigo_treino.category)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=200,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [ ]:
label_prevista = lr.predict(matriz_vetores_teste)
lr.score(matriz_vetores_teste, artigo_teste.category)

0.7964705308828548

In [ ]:
cr = classification_report(artigo_teste.category, label_prevista)
print(cr)

              precision    recall  f1-score   support

     colunas       0.86      0.72      0.78      6103
   cotidiano       0.61      0.79      0.69      1698
     esporte       0.93      0.88      0.90      4663
   ilustrada       0.13      0.88      0.23       131
     mercado       0.84      0.79      0.81      5867
       mundo       0.74      0.86      0.79      2051

    accuracy                           0.80     20513
   macro avg       0.68      0.82      0.70     20513
weighted avg       0.83      0.80      0.81     20513



### Dummy Classifier

In [ ]:
dc = DummyClassifier()
dc.fit(matriz_vetores_treino, artigo_treino.category)

/usr/local/lib/python3.7/dist-packages/sklearn/dummy.py:132: FutureWarning: The default value of strategy will change from stratified to prior in 0.24.
  "stratified to prior in 0.24.", FutureWarning)


DummyClassifier(constant=None, random_state=None, strategy='warn')

In [ ]:
label_prevista_dc = dc.predict(matriz_vetores_teste)
dc.score(matriz_vetores_teste, artigo_teste.category)

0.1672110369034271

In [ ]:
cr_dc = classification_report(artigo_teste.category, label_prevista_dc)
print(cr_dc)

              precision    recall  f1-score   support

     colunas       0.29      0.16      0.21      6103
   cotidiano       0.09      0.17      0.11      1698
     esporte       0.23      0.16      0.19      4663
   ilustrada       0.01      0.24      0.02       131
     mercado       0.29      0.17      0.21      5867
       mundo       0.11      0.19      0.14      2051

    accuracy                           0.17     20513
   macro avg       0.17      0.18      0.15     20513
weighted avg       0.24      0.17      0.19     20513



## Treinamento com Skip-gram

In [ ]:
modelo_skipgram = KeyedVectors.load_word2vec_format("skip_s300.txt")

In [ ]:
def combinacao_vetores_soma_skipgram(palavras_numeros):
  vetor_resultante = np.zeros(300)

  for pn in palavras_numeros:
    try:
      vetor_resultante += modelo_skipgram.get_vector(pn)
    except KeyError:
      if pn.isnumeric():
        pn = '0'*len(pn)
        vetor_resultante += modelo_skipgram.get_vector(pn)
      else:
        vetor_resultante += modelo_skipgram.get_vector('unknown')

  return vetor_resultante

def matriz_vetores_skipgram(textos):
  x = len(textos)
  y = 300
  matriz = np.zeros((x, y))

  for i in range(x):
    palavras_numeros = tokenizador(textos.iloc[i])
    matriz[i] = combinacao_vetores_soma_skipgram(palavras_numeros)
  
  return matriz

In [ ]:
matriz_vetores_treino_skipgram = matriz_vetores_skipgram(artigo_treino.title)
matriz_vetores_teste_skipgram = matriz_vetores_skipgram(artigo_teste.title)

In [ ]:
lr_skipgram = LogisticRegression(max_iter=300)
lr_skipgram.fit(matriz_vetores_treino_skipgram, artigo_treino.category)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=300,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [ ]:
label_prevista_skipgram = lr.predict(matriz_vetores_teste_skipgram)
lr_skipgram.score(matriz_vetores_teste_skipgram, artigo_teste.category)

0.807000438746161

In [ ]:
cr_skipgram = classification_report(artigo_teste.category, label_prevista_skipgram)
print(cr_skipgram)

              precision    recall  f1-score   support

     colunas       0.86      0.72      0.78      6103
   cotidiano       0.63      0.81      0.70      1698
     esporte       0.93      0.89      0.91      4663
   ilustrada       0.15      0.92      0.26       131
     mercado       0.84      0.82      0.83      5867
       mundo       0.76      0.86      0.81      2051

    accuracy                           0.81     20513
   macro avg       0.69      0.83      0.71     20513
weighted avg       0.84      0.81      0.82     20513

